<h1 style="text-align:center">Capstone Project - The Battle of Neighborhoods</h1>


In [4]:
import numpy as np
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
#%conda install wget
print("Library successfully imported")

Library successfully imported


In [33]:
df= pd.read_csv('https://data.lacounty.gov/api/views/c3xr-3jw2/rows.csv?accessType=DOWNLOAD')

In [34]:
df=df[['ZIP Code','Postal City 1','Location']]
df

,ZIP Code,Postal City 1,Location
0,90713,Lakewood,"90713(33.84871142900005, -118.11357922799999)"
1,91306,Winnetka,"91306(34.208404020000046, -118.57593995299999)"
2,90002,Los Angeles,"90002(33.94895070600006, -118.24697958699994)"
3,90506,Torrance,"90506(33.88535286100006, -118.32659746799999)"
4,90069,West Hollywood,"90069(34.08940300900008, -118.37978902499998)"
...,...,...,...
365,90011,Los Angeles,"90011(34.007903741000064, -118.259036977)"
366,90247,Gardena,"90247(33.89189209400007, -118.29849831699994)"
367,90601,Whittier,"90601(33.995472680000034, -118.04046581999995)"
368,90630,Cypress,"90630(33.82014301800007, -118.03980972399995)"


In [35]:
df['Coordinate']=df["Location"].apply(lambda st: st[st.find("(")+1:st.find(")")])
df

,ZIP Code,Postal City 1,Location,Coordinate
0,90713,Lakewood,"90713(33.84871142900005, -118.11357922799999)","33.84871142900005, -118.11357922799999"
1,91306,Winnetka,"91306(34.208404020000046, -118.57593995299999)","34.208404020000046, -118.57593995299999"
2,90002,Los Angeles,"90002(33.94895070600006, -118.24697958699994)","33.94895070600006, -118.24697958699994"
3,90506,Torrance,"90506(33.88535286100006, -118.32659746799999)","33.88535286100006, -118.32659746799999"
4,90069,West Hollywood,"90069(34.08940300900008, -118.37978902499998)","34.08940300900008, -118.37978902499998"
...,...,...,...,...
365,90011,Los Angeles,"90011(34.007903741000064, -118.259036977)","34.007903741000064, -118.259036977"
366,90247,Gardena,"90247(33.89189209400007, -118.29849831699994)","33.89189209400007, -118.29849831699994"
367,90601,Whittier,"90601(33.995472680000034, -118.04046581999995)","33.995472680000034, -118.04046581999995"
368,90630,Cypress,"90630(33.82014301800007, -118.03980972399995)","33.82014301800007, -118.03980972399995"


In [36]:
df=df[['ZIP Code','Postal City 1','Coordinate']]
df=pd.concat([df,df['Coordinate'].str.split(',',expand =True)],axis=1)
df=df[['ZIP Code','Postal City 1',0,1]]
df.rename(columns={0:'Latitude',1:'Longitude'},inplace= True)

In [37]:
df

,ZIP Code,Postal City 1,Latitude,Longitude
0,90713,Lakewood,33.84871142900005,-118.11357922799999
1,91306,Winnetka,34.208404020000046,-118.57593995299999
2,90002,Los Angeles,33.94895070600006,-118.24697958699994
3,90506,Torrance,33.88535286100006,-118.32659746799999
4,90069,West Hollywood,34.08940300900008,-118.37978902499998
...,...,...,...,...
365,90011,Los Angeles,34.007903741000064,-118.259036977
366,90247,Gardena,33.89189209400007,-118.29849831699994
367,90601,Whittier,33.995472680000034,-118.04046581999995
368,90630,Cypress,33.82014301800007,-118.03980972399995


In [44]:
df[['Latitude','Longitude']]=df[['Latitude','Longitude']].astype('float')

In [45]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [46]:
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['Postal City 1']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la